This is the file that makes all of the API calls and saves the information to a folder of csv files

In [ ]:
import googlemaps
from dotenv import load_dotenv
import os
import time
import pandas as pd
from tqdm.notebook import tqdm
import random

load_dotenv()

G_KEY = os.getenv('G_KEY')

gmaps = googlemaps.Client(key=G_KEY)

cities = pd.read_csv('data.csv')
cities

In [ ]:
def getResults(lat, long, token=None):
    if token != None:
        request = gmaps.places_nearby(location=[lat, long], radius=50000, type='restaurant', page_token=token)
    else:
        request = gmaps.places_nearby(location=[lat, long], radius=50000, type='restaurant')
    
    try:
        next_token = request['next_page_token']
    except:
        next_token = None
    
    results = request['results']
        
    return results, next_token

In [ ]:
def getCity(lat, long, name):
    t0 = time.time()

    print(f'Processing {name} ...', end =" ")
    
    if os.path.exists(f'cities/{name}.csv'):
        df = pd.read_csv(f'cities/{name}.csv')
        print(f'already exists, Mean = {round(df["rating"].mean(), 3)}')
        return
    
    final_results = []

    results, next_token = getResults(lat, long)
    final_results+=results
    
    for i in range(1, 30):
        time.sleep(2)
        results, next_token = getResults(lat, long, next_token)
        final_results+=results
        if next_token == None:
            break
            
            
    #Trying slightly different location
    
    differences = [-0.1, 0.1, -0.2, 0.2]
    for diff in differences :
        new_lat = lat+diff
        new_long = long+diff
        
        results, next_token = getResults(new_lat, new_long)
        final_results+=results
        
        for i in range(1, 30):
            time.sleep(2)
            results, next_token = getResults(new_lat, new_long, next_token)
            final_results+=results
            if next_token == None:
                break
    
    # Removing duplicates
    res = [i for n, i in enumerate(final_results) if i not in final_results[n + 1:]]

    df = pd.DataFrame(res)
    df.to_csv(f'cities/{name}.csv', index=False)
    
    t1 = time.time()
    print(f'Sucess with #{len(res)} results in {round(toal,2)} seconds. Mean = {round(df["rating"].mean(), 3)}')
        

In [ ]:


size = 331

# Iterate through table
for index, row in tqdm(cities[:size].iterrows(), total=size):
    lat = row['lat']
    long = row['long']
    name = row['city']
    getCity(lat, long, name)